In [1]:
import keras

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Softmax
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import numpy as np

In [14]:
import pandas as pd
df = pd.read_pickle('training_set.pkl')
df.head()

,idWlb,FM,S
0,82.0,82.0,95
1,82.0,82.0,95
2,82.0,82.0,95
3,82.0,82.0,95
4,82.0,82.0,95


In [23]:
import sklearn.preprocessing
lb = sklearn.preprocessing.LabelBinarizer()
lb.fit(range(9))

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [43]:

wells = [well for well in df.idWlb.drop_duplicates()]
formations = [int(f) for f in df.FM.drop_duplicates()]
print(formations)
blind_wells = [1486, 107, 1140, 6678, 874, 82, 1411, 3558, 5916, 4244]
training_wells = np.setdiff1d(wells, blind_wells)

print(len(blind_wells))
print(len(training_wells))

list_of_wells = [df.loc[df.idWlb == well,['S','FM']] for well in training_wells]
list_of_wells[0].head()

to_formation_id = np.vectorize(lambda x: formations.index(x))

input_sequences = [np.expand_dims(df['S'].values.astype(dtype=np.int), axis=1) for df in list_of_wells]
output_sequences = [lb.transform(to_formation_id(df['FM'].values.astype(dtype=np.int))+1) for df in list_of_wells]

[82, 63, 159, 118, 174, 46, 150, 83, 47]
10
43


In [44]:
batch_size = 1
time_step = 5
latent_dim=256

from keras.preprocessing.sequence import TimeseriesGenerator

training_set = []
for symbols, targets in zip(input_sequences, output_sequences):
    training_set.append(TimeseriesGenerator(symbols, targets,
                               length=time_step,
                               sampling_rate=1,
                               batch_size=batch_size))
print(len(training_set))

43


In [45]:
input = Input(shape=(time_step, 1))
lstm = LSTM(latent_dim,
            input_shape=(time_step, 1))(input)
output = Dense(9, activation="softmax")(lstm)

In [46]:
model = Model(input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 5, 1)              0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               264192    
_________________________________________________________________
dense_10 (Dense)             (None, 9)                 2313      
Total params: 266,505
Trainable params: 266,505
Non-trainable params: 0
_________________________________________________________________


In [49]:
checkpoint = ModelCheckpoint('./1xLSTM_intervals.best.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='max')

num_iter = 100
history = []

for i in range(0, num_iter):
    print("iter", i)
    for w, well in enumerate(training_set):
        print("well", w)
        history.append(model.fit_generator(well,
                      epochs=1,
                      callbacks=[checkpoint]))

iter 0
well 0
Epoch 1/1
1635/1635 [==============================] - 17s 10ms/step - loss: 0.4415

Epoch 00001: loss improved from -inf to 0.44146, saving model to ./1xLSTM_intervals.best.hdf5
well 1
Epoch 1/1
10757/10757 [==============================] - 109s 10ms/step - loss: 2.4721

Epoch 00001: loss improved from 0.44146 to 2.47210, saving model to ./1xLSTM_intervals.best.hdf5
well 2
Epoch 1/1
14468/14468 [==============================] - 145s 10ms/step - loss: 1.6833

Epoch 00001: loss did not improve from 2.47210
well 3
Epoch 1/1
8736/8736 [==============================] - 87s 10ms/step - loss: 0.0630

Epoch 00001: loss did not improve from 2.47210
well 4
Epoch 1/1
6229/6229 [==============================] - 62s 10ms/step - loss: 1.6180

Epoch 00001: loss did not improve from 2.47210
well 5
Epoch 1/1
14546/14546 [==============================] - 145s 10ms/step - loss: 0.5366

Epoch 00001: loss did not improve from 2.47210
well 6
Epoch 1/1
7219/7219 [=========================

5685/5685 [==============================] - 57s 10ms/step - loss: 1.2251

Epoch 00001: loss did not improve from 4.34625
well 14
Epoch 1/1
16163/16163 [==============================] - 162s 10ms/step - loss: 1.6762

Epoch 00001: loss did not improve from 4.34625
well 15
Epoch 1/1
7561/7561 [==============================] - 76s 10ms/step - loss: 1.7312

Epoch 00001: loss did not improve from 4.34625
well 16
Epoch 1/1
7155/7155 [==============================] - 72s 10ms/step - loss: 2.6283

Epoch 00001: loss did not improve from 4.34625
well 17
Epoch 1/1
2969/2969 [==============================] - 30s 10ms/step - loss: 1.4809

Epoch 00001: loss did not improve from 4.34625
well 18
Epoch 1/1
9294/9294 [==============================] - 94s 10ms/step - loss: 2.3176

Epoch 00001: loss did not improve from 4.34625
well 19
Epoch 1/1
2751/2751 [==============================] - 28s 10ms/step - loss: 5.4732

Epoch 00001: loss improved from 4.34625 to 5.47325, saving model to ./1xLSTM_inter

4935/4935 [==============================] - 50s 10ms/step - loss: 2.0653

Epoch 00001: loss did not improve from 15.19140
well 27
Epoch 1/1
9243/9243 [==============================] - 93s 10ms/step - loss: 0.6883

Epoch 00001: loss did not improve from 15.19140
well 28
Epoch 1/1
5063/5063 [==============================] - 51s 10ms/step - loss: 1.2814

Epoch 00001: loss did not improve from 15.19140
well 29
Epoch 1/1
10480/10480 [==============================] - 106s 10ms/step - loss: 1.7394

Epoch 00001: loss did not improve from 15.19140
well 30
Epoch 1/1
5484/5484 [==============================] - 55s 10ms/step - loss: 2.7218

Epoch 00001: loss did not improve from 15.19140
well 31
Epoch 1/1
9347/9347 [==============================] - 94s 10ms/step - loss: 1.7300

Epoch 00001: loss did not improve from 15.19140
well 32
Epoch 1/1
5500/5500 [==============================] - 55s 10ms/step - loss: 1.9297

Epoch 00001: loss did not improve from 15.19140
well 33
Epoch 1/1
12129/1212

6982/6982 [==============================] - 70s 10ms/step - loss: 1.6407

Epoch 00001: loss did not improve from 15.19140
well 42
Epoch 1/1
10767/10767 [==============================] - 108s 10ms/step - loss: 0.6766

Epoch 00001: loss did not improve from 15.19140
iter 4
well 0
Epoch 1/1
1635/1635 [==============================] - 16s 10ms/step - loss: 0.3497

Epoch 00001: loss did not improve from 15.19140
well 1
Epoch 1/1
10757/10757 [==============================] - 108s 10ms/step - loss: 4.6410

Epoch 00001: loss did not improve from 15.19140
well 2
Epoch 1/1
14468/14468 [==============================] - 145s 10ms/step - loss: 3.2460

Epoch 00001: loss did not improve from 15.19140
well 3
Epoch 1/1
8736/8736 [==============================] - 88s 10ms/step - loss: 0.0585

Epoch 00001: loss did not improve from 15.19140
well 4
Epoch 1/1
6229/6229 [==============================] - 63s 10ms/step - loss: 4.3393

Epoch 00001: loss did not improve from 15.19140
well 5
Epoch 1/1
145

well 13
Epoch 1/1
5685/5685 [==============================] - 57s 10ms/step - loss: 3.2217

Epoch 00001: loss did not improve from 15.19140
well 14
Epoch 1/1
16163/16163 [==============================] - 163s 10ms/step - loss: 1.7781

Epoch 00001: loss did not improve from 15.19140
well 15
Epoch 1/1
7561/7561 [==============================] - 76s 10ms/step - loss: 1.8182

Epoch 00001: loss did not improve from 15.19140
well 16
Epoch 1/1
7155/7155 [==============================] - 72s 10ms/step - loss: 3.4998

Epoch 00001: loss did not improve from 15.19140
well 17
Epoch 1/1
2969/2969 [==============================] - 30s 10ms/step - loss: 1.4674

Epoch 00001: loss did not improve from 15.19140
well 18
Epoch 1/1
9294/9294 [==============================] - 94s 10ms/step - loss: 2.4553

Epoch 00001: loss did not improve from 15.19140
well 19
Epoch 1/1
2751/2751 [==============================] - 28s 10ms/step - loss: 5.4807

Epoch 00001: loss did not improve from 15.19140
well 20
Ep

...

well 14
Epoch 1/1
16163/16163 [==============================] - 165s 10ms/step - loss: 8.9169

Epoch 00001: loss did not improve from 15.19140
well 15
Epoch 1/1
7561/7561 [==============================] - 77s 10ms/step - loss: 8.4549

Epoch 00001: loss did not improve from 15.19140
well 16
Epoch 1/1
7155/7155 [==============================] - 73s 10ms/step - loss: 5.1634

Epoch 00001: loss did not improve from 15.19140
well 17
Epoch 1/1
2969/2969 [==============================] - 30s 10ms/step - loss: 1.5087

Epoch 00001: loss did not improve from 15.19140
well 18
Epoch 1/1
9294/9294 [==============================] - 95s 10ms/step - loss: 7.3960

Epoch 00001: loss did not improve from 15.19140
well 19
Epoch 1/1
2751/2751 [==============================] - 28s 10ms/step - loss: 6.5194

Epoch 00001: loss did not improve from 15.19140
well 20
Epoch 1/1
43/43 [==============================] - 0s 10ms/step - loss: 0.2328

Epoch 00001: loss did not improve from 15.19140
well 21
Epoch 1

well 29
Epoch 1/1
10480/10480 [==============================] - 107s 10ms/step - loss: 7.6247

Epoch 00001: loss did not improve from 15.19140
well 30
Epoch 1/1
5484/5484 [==============================] - 56s 10ms/step - loss: 2.7356

Epoch 00001: loss did not improve from 15.19140
well 31
Epoch 1/1
9347/9347 [==============================] - 95s 10ms/step - loss: 4.7083

Epoch 00001: loss did not improve from 15.19140
well 32
Epoch 1/1
5500/5500 [==============================] - 56s 10ms/step - loss: 8.7304

Epoch 00001: loss did not improve from 15.19140
well 33
Epoch 1/1
12129/12129 [==============================] - 124s 10ms/step - loss: 5.1889

Epoch 00001: loss did not improve from 15.19140
well 34
Epoch 1/1
11538/11538 [==============================] - 117s 10ms/step - loss: 4.2100

Epoch 00001: loss did not improve from 15.19140
well 35
Epoch 1/1
6271/6271 [==============================] - 64s 10ms/step - loss: 6.2947

Epoch 00001: loss did not improve from 15.19140
well

well 1
Epoch 1/1
10757/10757 [==============================] - 110s 10ms/step - loss: 7.9958

Epoch 00001: loss did not improve from 15.19140
well 2
Epoch 1/1
14468/14468 [==============================] - 147s 10ms/step - loss: 5.6128

Epoch 00001: loss did not improve from 15.19140
well 3
Epoch 1/1
8736/8736 [==============================] - 89s 10ms/step - loss: 0.1474

Epoch 00001: loss did not improve from 15.19140
well 4
Epoch 1/1
6229/6229 [==============================] - 63s 10ms/step - loss: 5.0604

Epoch 00001: loss did not improve from 15.19140
well 5
Epoch 1/1
14546/14546 [==============================] - 148s 10ms/step - loss: 3.5299

Epoch 00001: loss did not improve from 15.19140
well 6
Epoch 1/1
7219/7219 [==============================] - 74s 10ms/step - loss: 7.2515

Epoch 00001: loss did not improve from 15.19140
well 7
Epoch 1/1
13184/13184 [==============================] - 134s 10ms/step - loss: 0.8739

Epoch 00001: loss did not improve from 15.19140
well 8
E

well 16
Epoch 1/1
7155/7155 [==============================] - 73s 10ms/step - loss: 5.1622

Epoch 00001: loss did not improve from 15.19140
well 17
Epoch 1/1
2969/2969 [==============================] - 30s 10ms/step - loss: 1.4988

Epoch 00001: loss did not improve from 15.19140
well 18
Epoch 1/1
9294/9294 [==============================] - 94s 10ms/step - loss: 7.4045

Epoch 00001: loss did not improve from 15.19140
well 19
Epoch 1/1
2751/2751 [==============================] - 28s 10ms/step - loss: 6.5241

Epoch 00001: loss did not improve from 15.19140
well 20
Epoch 1/1
43/43 [==============================] - 0s 11ms/step - loss: 0.2209

Epoch 00001: loss did not improve from 15.19140
well 21
Epoch 1/1
2865/2865 [==============================] - 29s 10ms/step - loss: 4.6759

Epoch 00001: loss did not improve from 15.19140
well 22
Epoch 1/1
6901/6901 [==============================] - 70s 10ms/step - loss: 5.6474

Epoch 00001: loss did not improve from 15.19140
well 23
Epoch 1/1


well 31
Epoch 1/1
9347/9347 [==============================] - 96s 10ms/step - loss: 4.7080

Epoch 00001: loss did not improve from 15.19140
well 32
Epoch 1/1
5500/5500 [==============================] - 56s 10ms/step - loss: 8.7371

Epoch 00001: loss did not improve from 15.19140
well 33
Epoch 1/1
12129/12129 [==============================] - 124s 10ms/step - loss: 5.1889

Epoch 00001: loss did not improve from 15.19140
well 34
Epoch 1/1
11538/11538 [==============================] - 117s 10ms/step - loss: 4.2132

Epoch 00001: loss did not improve from 15.19140
well 35
Epoch 1/1
6271/6271 [==============================] - 64s 10ms/step - loss: 6.2962

Epoch 00001: loss did not improve from 15.19140
well 36
Epoch 1/1
4957/4957 [==============================] - 51s 10ms/step - loss: 13.5109

Epoch 00001: loss did not improve from 15.19140
well 37
Epoch 1/1
4339/4339 [==============================] - 44s 10ms/step - loss: 9.0603

Epoch 00001: loss did not improve from 15.19140
well 3

In [51]:
# model.save('lstm_100iter_step5.hdf5')
model = keras.models.load_model('lstm_100iter_step5.hdf5')

In [52]:
list_of_blind_wells = [df.loc[df.idWlb == well,['S','FM']] for well in blind_wells]
list_of_blind_wells[0].head()


predict_input = [np.expand_dims(df['S'].values.astype(dtype=np.int), axis=1) for df in list_of_blind_wells]
predict_target = [np.expand_dims(to_formation_id(df['FM'].values.astype(dtype=np.int)), axis=1) for df in list_of_blind_wells]

In [53]:
from keras.preprocessing.sequence import TimeseriesGenerator
inference_set = []
for symbols, targets in zip(predict_input, predict_target):
    inference_set.append(TimeseriesGenerator(symbols, targets,
                               length=time_step,
                               sampling_rate=1,
                               batch_size=batch_size))

In [54]:
import math
results = []
for w, well_data in enumerate(inference_set):
    print(w, len(well_data))
    result = []
    for n in range(0, len(well_data)):
        symbols, _ = well_data[n]
        r = model.predict(symbols)[0][0]
        f = formations[math.floor(r+0.5)]
        result.append(f)   
    results.append(result)
    
# print(results)

0 6996
1 15573
2 5381
3 7051
4 10644
5 3484
6 11274
7 4266
8 13280
9 2935


In [55]:
print(blind_wells)

[1486, 107, 1140, 6678, 874, 82, 1411, 3558, 5916, 4244]


In [56]:
import gzip
import cloudpickle as pickle
with gzip.GzipFile('lstm_1000iter_step200.gzip', 'w') as fs:
        pickle.dump(results,fs)
        pickle.dump(blind_wells,fs)
